# Problems with text as tables (draft)

## State of research on CSV files

CSV used to stand for "***comma*** separated values", however recent research done by [Mitlöhner et.al](ref/mitl-etal-2016OBD.pdf),
concludes that it is more appropriate to to refer to CSV as "***character*** separated values", which echoes
concerns expressed by [Shafranovich, Y., 2005.](https://www.hjp.at/doc/rfc/rfc4180.html#sec_1) in the RFC4180:
"Common format and MIME type for comma-separated values (CSV) files."

In research prior to 2016 reading CSV files has been treated as a parsing problem.
2016 changed the approach, where it is being treated as a pattern recognition problem.

Between 2016 and 2018 the W3C consortium ran a work group who recommended CSV files
to be accompanied by a JSON which declares the dialect required to interpret the CSV
and required the CSV and JSON to be encoded in UTF-8.

A few implementations exist, such as [Frictionless (Java)](https://github.com/frictionlessdata/specs/tree/master/csv-dialect),
XXX a number of university projects xXXX, but no PyPi packages exist to read the dialects json.
Should the dialect file exist, the user will be required to interpret it and translate it into, python
`kwargs`.

Furthermore, the requirement to use UTF-8 has not been adopted in the wild. The consequence hereof is
several conversations in various forums why CSV readers can't read the data, where the reality is that
interpretation of CSV files requires additional knowledge about the encoding as this "standard W3C
recommendation" can't be taken for granted.

Finally, scraping of code on github reveals that CSV files frequently are compressed immediately after
creation. Following the thinking mental model that Python libraries exist for relieving the developer
of trivial matters, this practice should probably be considered. The W3C CSV work group recommends this
practice as it makes the CSV file and dialect accompany one another.

Last, but not least, most CSV writers do not produce the two files on output, nor permit the output to
be compressed.

## API

The requirements for pandas, would be:

[1] To ***read*** W3C defined CSV given pathlib.Path(s).

- If a single Path is given and the path.endswith `.zip`, the archive is decompressed with expectations to
find a single data and dialect file.

- If two paths are given, ending with `.csv` and `.json`, the csv can be read using the dialect file.


[2] To ***write*** W3C defined CSV given a pandas table and pathlib.Path(s).

- If a single path.name is given and ends with:

  -`zip` a compressed file is made with both data and dialect.

  -`csv` a file is made with data.

  -`json` a file is made with the dialect.

- If two paths are given and each end with `.csv` and `.json` two files are created with respectively
the data and the dialect.

[3] To read pathlib.Path or IoStream without dialect file and automatically detect the dialect.

  - This permits pandas to create a dialect file and use it for interpreting the data.
  - It is important to recognise the separation of concerns between the dialect detection
    and reading the data.
  - The detection will apply the practices developed by [Gerrit J.J. van den Burg, et.al](https://arxiv.org/pdf/1811.11242.pdf)
described further below.


### Estimating encoding and dialect


Conversations in forums extend the CSV problem to include encoding as the W3C recommendation

Existing csv-readers and file sniffers fail because they:
- presume US-ASCII or don't detect correct UTF+ era encoding.
- attempt to parser based on incorrect encoding.
- continue parsing despite that the format isn't recognised as tabular / rectangular.
  - Documents that hold more than one table which are separated by repeated line breaks.
  - Attempt to maintain a number of columns despite that the required number of delimiters exceed the rectangular format.
- Presume no or one line in the header, despite that multiline headers are detected in the wild.
- skip the last line because it is missing a newline character (optional in RFC4180)
- don't handle text escapes during column detection (RFC4180 expectation).
- presume that text escapes are based on double qoutes `"` and ignore commonly used brackets.
- dump leading zeros when inferring integer values.
- omitting empty columns and other [rules](https://cchristodoulaki.github.io/Pytheas/)
- usage of field detection like `dateutils` where the parser only evaluates a single value and will
  always say that 01.02.2012 is mm.dd.yyyy despite that the next value might be 29.2.2012 (I've got plenty of tests with that problem).

> file = [header CRLF] record *(CRLF record) [CRLF]
> header = name *(COMMA name)
> record = field *(COMMA field)
> name = field
> field = (escaped / non-escaped)
> escaped = DQUOTE *(TEXTDATA / COMMA / CR / LF / 2DQUOTE) DQUOTE
> non-escaped = *TEXTDATA
> COMMA = %x2C
> CR = %x0D ;as per section 6.1 of RFC 2234 [2]
> DQUOTE =  %x22 ;as per section 6.1 of RFC 2234 [2]
> LF = %x0A ;as per section 6.1 of RFC 2234 [2]
> CRLF = CR LF ;as per section 6.1 of RFC 2234 [2]
> TEXTDATA =  %x20-21 / %x23-2B / %x2D-7E


Here's an ugly `example-1`:

`Birthdate, (Family\nnames), Father, Mother, Child, known for\n1879-4-14, Einstein, Hermann, Pauline, Albert,"\nGeneral relativity,\nSpecial relativity,\nPhotoelectric effect"`

That should produce `table-1` (including some text escape):

|Birthdate| (Family<br>names)| Father | Mother | Child | known for |
|---|---|---|---|---|---|
|1879-4-14| Einstein| Hermann| Pauline| Albert| General relativity,<br>Special relativity,<br>Photoelectric effect|


Data sources:

- https://open.canada.ca/en
- the turing institute
- [csv_files from Mitlöhner et.al](refs/csv_files.csv) and [https://archiver.ai.wu.ac.at/](https://archiver.ai.wu.ac.at/)
- proprietary and confidential data (undisclosed, 1.5Tb)

Perspectives:
- https://github.com/cchristodoulaki/Pytheas
- https://www.gov.uk/government/publications/recommended-open-standards-for-government/using-metadata-to-describe-csv-data
- https://data.wu.ac.at/csvengine/
- https://github.com/w3c/csvw


### The ground truth.

To determine the ground truth, we do the following:

1. Assert the ability to generate csv from config and reconstruct config from csv.
2. Assert the ability to read CSVs "from the wild" and provide a clear declaration of errors.
3. Generate CSV files using various implementations, python, etc. and reconstruct the dialect.


arxiv.x uses a "score" to determine best match. I prefer that score to be entropy:

```python
def entropy(value_counts):
    """ Input is a db of value counts e.g. {True: 10, False:100, NULL: 5} """
    # Compute the shannon entropy of a column
    size = sum(value_counts.values())
    h_entropy = 0.0
    for _, count in value_counts.items():
        proportion = (count/size)
        h_entropy -= proportion * math.log(proportion, 2)
    return h_entropy
```


### real world dirt - [source](https://arxiv.org/pdf/1811.11242.pdf)
```
functions,,:stop,"a[u:stop,i]"
functions,,:stop,a[u:stop:b]
hotshot,,:lineno,"ncalls tottime"
httplib,,:port,host:port
imaplib,,:MM,"DD-MM-YY"
```

```
~437~^~a~^~Approve~^~3~^~13482~
~688~^~b~^~Jellyfish~^~1~^~12880~
~498~^~c~^~Color~^~2~^~13629~
~992~^~a~^~Wind~^~8~^~12392~
~246~^~c~^~Coat~^~0~^~13764~
```

```
"www.google.com,search,02/02/15"
"www.yahoo.com,search,02/02/15"
"www.bing.com,search,03/02/15"
"altavista.com,search,03/02/15"
"askjeeves.com,search,03/06/15"
```

```
#Release 0.4
#Copyright (c) 2015 SomeCompany.
#
Z10,,,HFJ,,,,,,
B12,,IZOY,AB_K9Z_DD_18,RED,,12,,,
```

```
Mango; £365,14; £1692,64
Apple; £2568,62; £1183,78
Lemon; £51,65; £685,67
Orange; £1760,75; £128,14
Maple; £880,86; £323,43
```


```
this, is, a file,"
with a number of issues
that shows ""double quoting""
\"escaping\" and multi-line cells
",\, and has only one row!
```

### Keys & properties.

The CSV readers that I've encountered are mainly parsers that hope to apply a pattern which then is produces
a valid table with data. The inputs for the `csv-[reader/parser/...]` are:

| keyword | pandas |
|---|---|
|filepath_or_buffer|filepath_or_buffer|
|sep|sep|
|delimiter|delimiter|
|header|header|
|names|names|
|index_col|index_col|
|usecols|usecols|
|squeeze|squeeze|
|prefix|prefix|
|mangle_dupe_cols|mangle_dupe_cols|
|dtype|dtype|
|engine|engine|
|converters|converters|
|true_values|true_values|
|false_values|false_values|
|skipinitialspace|skipinitialspace|
|skiprows|skiprows|
|skipfooter|skipfooter|
|nrows|nrows|
|na_values|na_values|
|keep_default_na|keep_default_na|
|na_filter|na_filter|
|verbose|verbose|
|skip_blank_lines|skip_blank_lines|
|parse_dates|parse_dates|
|infer_datetime_format|infer_datetime_format|
|keep_date_col|keep_date_col|
|date_parser|date_parser|
|dayfirst|dayfirst|
|cache_dates|cache_dates|
|iterator|iterator|
|chunksize|chunksize|
|compression|compression|
|thousands|thousands|
|decimal|decimal|
|lineterminator|lineterminator|
|quotechar|quotechar|
|quoting|quoting|
|doublequote|doublequote|
|escapechar|escapechar|
|comment|comment|
|encoding|encoding|
|encoding_errors|encoding_errors|
|dialect|dialect|
|error_bad_lines|error_bad_lines|
|warn_bad_lines|warn_bad_lines|
|on_bad_lines|on_bad_lines|
|delim_whitespace|delim_whitespace|
|low_memory|low_memory|
|memory_map|memory_map|
|float_precision|float_precision|
|storage_options|storage_options|

When given great attention to details, it becomes obvious that the inputs to `pandas` will not
suffice to convert `example-1` into `table-1`

The goal of this project is to analyze any csv file and return a set of `**kwargs` required to read the csv file.


### A solution?

To enable the user to do the following:

```
import pandas as pd
import csv_analyze
from pathlib import Path

df = pandas.csv_reader(somepath, **analyze(Path(s3://a_file.csv)))
```

This requires that:

- The encoding using one of the ~300 python accepted [encodings](https://github.com/Ousret/charset_normalizer/blob/51af624b59a7f1a1aaa36f9ae71bee4364e39409/charset_normalizer/constant.py#L310)
- analyse the file,
- return the correct formats to pandas as **kwargs

The output will contain all the required information to consume the csv file, including
more detailed information from the analysis:

```
d = csv.analyze(path)
d
    {"encoding": {"depth": 10043,  # characters checked.
                  'cp855': 10043,  # meaning 10043/10043 = 100% characters match.
                  'utf_8_sig': 12,  # meaning decode error after 12th character.
                  'utf-8': 5,
                  "....."},
     "text_escape": [False, True, False, False, True],
     "column_names": ["Birthdate", "(Family\nnames)", "Father", "Mother ", "Child", "known for"],
     "datatypes": ["date", "str", "str", "str", "str"],
     "metadata": [  # in order like columns, with list of probabilities for each type.
         {"date": {'yyyy-mm-dd': (100, 100),
                   'mm-dd-yyyy': (0, 100),
                   "....": (0, 0)},
          "time": 0,
          "datetime": 0,
          "str": "pass",
          "int": 0,
          "float": 0},
         {"date": ValueError, "time": ValueError, "datetime": ValueError, "str": (100, 100), "int": ValueError,
          "float": ValueError},
         {"date": ValueError, "time": ValueError, "datetime": ValueError, "str": (100, 100), "int": ValueError,
          "float": ValueError},
         {"date": ValueError, "time": ValueError, "datetime": ValueError, "str": (100, 100), "int": ValueError,
          "float": ValueError},
         {"date": ValueError, "time": ValueError, "datetime": ValueError, "str": (100, 100), "int": ValueError,
          "float": ValueError}
     ]
     }
```

## What is required to recognize patterns in a csv file?

5 properties must be established to recognize the pattern of a csv file:

- Encoding.
- Header / Footer information
- Rectangular format which may not be carriage return / line break character
- Field separation
- Field interpretation:
    - text escape
    - datatype which may be a local text, date or number system


### Encoding

### Header / footer information

### Rectangular format

### Field separation

### Field interpretation


The strings I've encountered in csv data are:

SEPARATORS

```
#,###.#####  last non-digit character indicates decimal, preceding different characters are thousand separators.
#.###,#####
```

Examples:
```
4 294 967 295,000  Canadian (English and French), Danish, Finnish, French, German
4.294.967.295,000  Italian, Norwegian, Spanish,
4 294 967 295,000  Swedish
4,294,967,295.000  GB-English, US-English, Thai
```
(full treaty on: https://en.wikipedia.org/wiki/Decimal_separator)

However, notice that Hindi uses a 2-digit grouping, except for the 3-digit grouping for denoting hundreds: 12,34,56,789.00

SCIENTIFIC NOTATION

```
   ###E###   integer before and after E
   ###e###
#.####E###   floating point before, integer after E
#.####e###
```

```
###.###N <=3 digit float, followed by N belongs to (K,M,G/B, T,E,P) for kilo, mega, giga/bill, tera, exa,...
```

POSITIVE/NEGATIVE

Negative numbers can have tailing minus
```
527-
-527
(527)
[527]
```

ADDITIONAL SIGNS

The same applies for percentages: 98%, 98 %, 98 pct, %98
And for currencies: $100.00, kr100,00

NON-LATIN numbers

NUMBER FORMATTING

    Script	Digits Used
    Latin	0 1 2 3 4 5 6 7 8 9
    Arabic	٠‎ ١‎ ٢‎ ٣‎ ٤‎ ٥‎ ٦‎ ٧‎ ٨‎ ٩
    Chinese / Japanese	〇 一 二 三 四 五 六 七 八 九 十…
    Hebrew	א ,ב ,ג, ד, ה, ו, ז, ח ,ט…
    Korean	일 이 삼 사 오 육 칠 팔 구…
    The Korean regularly uses both a Sino-Korean system and a native Korean system. Everything that can be counted will use one of the two systems, but seldom both.	하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉….
    Bengla	০ ১ ২ ৩ ৪ ৫ ৬ ৭ ৮ ৯
    Devanagari (script used to write Hindi,Marathi, and other languages)	० १ २ ३ ४ ५ ६ ७ ८ ९
    Gujarati	૦ ૧ ૨ ૩ ૪ ૫ ૬ ૭ ૮ ૯
    Gurmukhi (one of the scripts used to write Punjabi)	੦ ੧ ੨ ੩ ੪ ੫ ੬ ੭ ੮ ੯
    Kannada	೦ ೧ ೨ ೩ ೪ ೫ ೬ ೭ ೮ ೯
    Malayalam	൦ ൧ ൨ ൩ ൪ ൫ ൬ ൭ ൮ ൯
    Odia	୦ ୧ ୨ ୩ ୪ ୫ ୬ ୭ ୮ ୯
    Tamil	௦ ௧ ௨ ௩ ௪ ௫ ௬ ௭ ௮ ௯
    Telugu	౦ ౧ ౨ ౩ ౪ ౫ ౬ ౭ ౮ ౯
    Thai	๐ ๑ ๒ ๓ ๔ ๕ ๖ ๗ ๘ ๙
    Tibetan	༠ ༡ ༢ ༣ ༤ ༥ ༦ ༧ ༨ ༩

FLOATING POINT
the floating point precision issue (which in particular haunts anyone who reads long barcodes that aren't imported as integers):
```
val = "0.3066101993807095471566981359501369297504425048828125"
print(float(val))
0.30661019938070955
```

NOT A NUMBER
```
"", "#N/A", "#N/A N/A", "#NA", "-1.#IND", "-1.#QNAN", "-NaN", "-nan", "1.#IND", "1.#QNAN", "<NA>", "N/A", "NA", "NULL", "NaN", "n/a", "nan", "null". "-", "--", "###"
```

Similar variation appears in datetime locale:

```
yyyy-mm-dd  Canadian (English and French), Danish, German, Swedish
dd.mm.yyyy  Finnish
dd.mm.yy    Italian, Norwegian
dd-mm-yy    Spanish
dd/mm/yy    GB-English
mm-dd-yy    US-English
dd/mm/yyyy  Thai
```

And so for time:
```
23:59      Canadian
23.59      Finnish
23.59 Uhr  German
Kl 23.59   Norwegian
11:59 PM   Thai
11.59 PM   UK english
```

TEXT ESCAPE
Finally we also see newline characters in headers which the csv-reader cannot deal with. To detect the correct format of the example below multiple lines have to be read and the internal between newline and separators. These will also have to be text and bracket escaped.

`Birthdate, (Family\nnames), Father, Mother, Child, known for\n1879-4-14, Einstein, Hermann, Pauline, Albert,"\nGeneral relativity,\nSpecial relativity,\nPhotoelectric effect"`

See table in introduction as `ugly example`

LINEBREAK
very old files can have linebreaks \r \r\n or \n

### Data sources:

Here is a [list with 1400 text based tabular files](https://gist.github.com/root-11/0cb2b328b313a7d269e28d4083a6a726)


